In [28]:
from dataclasses import dataclass
import dataclasses
import struct
import random
import socket

# 1.1 DNSHeader와 DNSQuestion 클래스 작성하기

DNS Header는 아래와 같은 정보를 가지고 있다.
- query ID
- flags
- 4개의 count (num_questions, num_answers, num_authorities, num_additionals)는 DNS 패킷의 각 섹션에 얼마나 많은 레코드가 있는지 알려준다.

In [3]:
@dataclass
class DNSHeader:
    id: int
    flags: int
    num_questions: int = 0
    num_answers: int = 0
    num_authorities: int = 0
    num_additionals: int = 0

DNS Question은 3개의 필드를 가지고 있다.
- name (예: example.com)
- type (예: A record)
- class (항상 같음)

In [4]:
@dataclass
class DNSQuestion:
    name: bytes
    type_: int
    class_: int

# 1.2 bytes로 변환하기

In [5]:
def header_to_bytes(header):
    fields = dataclasses.astuple(header)
    # there are 6 `H`s because there are 6 fields
    return struct.pack("!HHHHHH", *fields)

def question_to_bytes(question):
    return question.name + struct.pack("!HH", question.type_, question.class_)

strunct.pack은 데이터를 바이트로 변환하는 함수이다. `!`는 네트워크 byte order를 의미하고, `H`는 2byte integer를 의미한다. C의 unsigned short와 같다. [참고](https://docs.python.org/ko/3/library/struct.html#format-characters)  
아래의 예시를 살펴보자.

In [7]:
struct.pack('!HH', 5, 23)

b'\x00\x05\x00\x17'

HH는 2개의 2byte integer를 의미한다. 따라서 5와 23이 각각 2byte로 변환된다.

In [8]:
print("big endian: ", struct.pack('!I', 16909060))
print("little endian: ", struct.pack('I', 16909060))

big endian:  b'\x01\x02\x03\x04'
little endian:  b'\x04\x03\x02\x01'


- big endian -> 사람이 숫자를 쓰는 방법과 같이 큰 단위의 바이트가 앞에 오는 방법
- littlen endian -> 작은 단위의 바이트가 앞에 오는 방법

네트워크 패킷에서는 정수가 항상 big endian 방식으로 인코딩된다. 그러나 다른 상황에서는 little endian이 기본값이다. 따라서 `!`는 "컴퓨터 네트워킹을 위한 바이트 순서를 사용하라"는 의미이다. 

# 1.3 name 인코딩

DNS query를 하기 위해선 도메인 이름 또한 인코딩해야 한다. 인코딩하는 방법은 아래와 같다.
- 문자열을 ASCII로 인코딩한다.
- 도메인 이름을 `.`을 기준으로 분리한다.
- 분리된 도메인 이름의 앞에 각 도메인 이름의 길이를 붙인다.
  - 예를 들면, `naver`는 `\x05naver`로 변환된다.
  - 각 레이블의 길이는 1바이트로 표현된다.
- 변환한 도메인을 모두 합친다.
- 도메인 이름의 끝에 `0`을 붙인다.

In [20]:
def encode_dns_name(domain_name):
    encoded = b""
    for part in domain_name.encode("ascii").split(b"."):
        encoded += bytes([len(part)]) + part
    return encoded + b"\x00"

# 1.4 쿼리 생성하기

In [26]:
random.seed(1)

TYPE_A = 1
CLASS_IN = 1

def build_query(domain_name, record_type):
    name = encode_dns_name(domain_name)
    id_ = random.randint(0, 65535)
    recursion_desired = 1 << 8
    header = DNSHeader(id=id_, num_questions=1, flags=recursion_desired)
    question = DNSQuestion(name=name, type_=record_type, class_=CLASS_IN)
    return header_to_bytes(header) + question_to_bytes(question)

build_query(("blog.horang.dev"), TYPE_A)

b'D\xcb\x01\x00\x00\x01\x00\x00\x00\x00\x00\x00\x04blog\x06horang\x03dev\x00\x00\x01\x00\x01'

1. RFC 1035에서 section 3.2.2 to 3.2.4를 참고하여 query type과 class를 정의한다.
2. encode_dns_name으로 DNS 이름을 인코딩한다.
3. query를 위한 랜덤 ID를 생성한다.
4. flag를 "recursion desired"로 설정한다. 이는 DNS resolver와 통신할 때 설정해야 한다. flag의 인코딩은 RFC 1035의 section 4.1.1에 정의되어 있다. RECURSION_DESIRED = 1<<8의 이유는 RFC 1035에 따르면 Recursion Desired bit는 flags 필드에서 오른쪽에서 9번째 비트이다.(1<<8 = 100000000)
5. question을 생성한다.
6. header와 question을 합친다.

# 1.5 테스트

In [41]:
query = build_query("blog.horang.dev", TYPE_A)

# create a UDP socket
# `socket.AF_INET` means that we're connecting to the internet
#                  (as opposed to a Unix domain socket `AF_UNIX` for example)
# `socket.SOCK_DGRAM` means "UDP"
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# send our query to 8.8.8.8, port 53. Port 53 is the DNS port.
sock.sendto(query, ("8.8.8.8", 53))

# read the response. UDP DNS responses are usually less than 512 bytes
# (see https://www.netmeister.org/blog/dns-size.html for MUCH more on that)
# so reading 1024 bytes is enough
response, _ = sock.recvfrom(1024)
print(response)
print(response.hex())

b'\xe6#\x81\x80\x00\x01\x00\x05\x00\x00\x00\x00\x04blog\x06horang\x03dev\x00\x00\x01\x00\x01\xc0\x0c\x00\x05\x00\x01\x00\x008@\x00\x13\x07chl8469\x06github\x02io\x00\xc0-\x00\x01\x00\x01\x00\x00\x0e\x10\x00\x04\xb9\xc7l\x99\xc0-\x00\x01\x00\x01\x00\x00\x0e\x10\x00\x04\xb9\xc7m\x99\xc0-\x00\x01\x00\x01\x00\x00\x0e\x10\x00\x04\xb9\xc7n\x99\xc0-\x00\x01\x00\x01\x00\x00\x0e\x10\x00\x04\xb9\xc7o\x99'
e6238180000100050000000004626c6f6706686f72616e67036465760000010001c00c000500010000384000130763686c383436390667697468756202696f00c02d0001000100000e100004b9c76c99c02d0001000100000e100004b9c76d99c02d0001000100000e100004b9c76e99c02d0001000100000e100004b9c76f99


다음 단계에서 이 response를 parsing하는 방법을 살펴볼 것이다.